# Combining Results for Multiple Days

This notebook outlines the code for consolidating optimal results across multiple days. The process involves assigning a score to each location based on its frequency of occurrence. Additionally, there is the option to multiply this score by the probability of occurrence for a specific day. The ultimate objective is to select a predefined number of road inspectors from the locations with the highest scores, while adhering to specific constraints.

To assess the performance of these selected locations, an approach is used where each location is mirrored on the opposite side of the road. This mirroring is essential as the road network lacks data to easily navigate off and on ramps for travel in the opposite direction.

In the final step, the optimization model is executed, using only the selected locations as candidates instead of all candidate locations. The primary purpose is to calculate the performance of these chosen locations, offering insights into the efficiency and effectiveness of the road inspectors locations.


In [1]:
pip install pygeos

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import functions
import folium
import pandas as pd 
import geopandas as gpd 
import numpy as np
import networkx as nx
import ast
from fuzzywuzzy import fuzz
from datetime import datetime
import matplotlib.pyplot as plt 
import math
import shapely.wkt
import time
from gurobipy import *
from functions import DutchRDtoWGS84
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
%matplotlib notebook

C:\Users\Klaas Sicking\Documents\studie\studie\Aerospace\advanced data science\Pro\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Data Preparation

This section involves loading and preparing various data sources. It includes reading candidate locations, weights, road network, and incident data.



In [4]:
unique_candidates = pd.read_csv('input/unique_candidates.csv')
df = pd.read_csv('input/weight.csv')
gdf = gpd.read_file('input/Snelheid_Wegvakken.shp')
road_network = gdf
G = nx.read_graphml("input/highway_nodes_DiGraph_nb.xml")
all_incidents = pd.read_csv("input/Incidents_clean.csv")
all_incidents['starttime_new'] = pd.to_datetime(all_incidents['starttime_new'])

df_without_weight = pd.read_csv('input/result.csv')
sums_without_weight = df_without_weight.drop(['Date'], axis=1).sum()
df_with_weight = pd.read_csv('input/weight.csv')
sums_with_weight = df_with_weight.drop(['Date', 'Event_Count'], axis=1).sum()

In [5]:
def process_and_save_files(input_folder, output_file):
    """
    Process and aggregate the optimisation results from multiple days.

    Args:
        input_folder (str): Path to the folder containing input files.
        output_file (str): Path to the output CSV file.

    Returns:
        pd.DataFrame: DataFrame containing aggregated data for all files.

    This function reads and processes data from multiple files in the input folder. Each file is expected to have
    columns 'X', 'Y', and 'Index'. The 'Index' column is used to aggregate data into a single DataFrame where
    each row represents a file. The resulting DataFrame is saved to the output CSV file.

    """
    file_list = os.listdir(input_folder)
    all_days = pd.DataFrame(columns=range(0, 1936))

    for file in file_list:
        file_path = os.path.join(input_folder, file)
        df = pd.read_csv(file_path, sep=" ", header=None, names=['X', 'Y', 'Index'])
        new_row = [0] * 1936

        for index_val in df.iloc[:, 2]:
            idx = int(index_val)
            new_row[idx] = 1

        all_days.loc[file] = new_row

    all_days.to_csv(output_file, index_label='Date')
    return all_days


In [6]:
# Example usage of the function
input_folder = "input/output_files/"
output_file = 'result/result.csv'
result_df = process_and_save_files(input_folder, output_file)

## Aggregating Optimization Results for Road Inspector Locations

This code snippet processes and aggregates optimal road inspector locations obtained from multiple days. It reads data from a folder containing several files, each representing the optimal locations for road inspectors on a specific day. Each file is expected to have columns 'X', 'Y,' and 'Index.' The 'Index' column is used to consolidate the data into a single DataFrame, where each row represents the optimal locations for road inspectors on one day. The resulting DataFrame is then saved to an output CSV file.


In [7]:
def get_weighted_results(pdf_file, result_file, output_file):
    """
    Combine and apply weights to result data using probabilities from a PDF file.

    Parameters:
        pdf_file (str): Path to the PDF data CSV file containing the probabilities for each date.
        result_file (str): Path to the result data CSV file with results for various dates.
        output_file (str): Path to the output CSV file to save the combined and weighted results.

    Returns:
        pd.DataFrame: Processed and merged DataFrame with weighted values.
    """
    # Read the PDF data and format the date column
    pdf_df = pd.read_csv(pdf_file)
    pdf_df['Date'] = pd.to_datetime(pdf_df['Date'], infer_datetime_format=True)

    # Read the result data and format the date column
    result_df = pd.read_csv(result_file)
    result_df['Date'] = pd.to_datetime(result_df['Date'], infer_datetime_format=True)

    # Merge the DataFrames
    merged_df = pd.merge(result_df, pdf_df, on='Date', how='inner')

    # Calculate weighted values
    columns_to_multiply = [col for col in result_df.columns if col != 'Date']
    for col in columns_to_multiply:
        merged_df[col] = merged_df[col] * merged_df['PDF_Value']

    # Save the result to a new CSV file
    merged_df.drop('PDF_Value', axis=1, inplace=True)
    merged_df.to_csv(output_file, index=False)

    return merged_df

In [8]:
pdf_file = 'result/pdf.csv'
result_file = 'result/result.csv'
output_file = 'result/weight.csv'
weights = get_weighted_results(pdf_file, result_file, output_file)
weights = df_with_weight.drop(['Date', 'Event_Count'], axis=1).sum()

## Data Transformation for Desired Format

In order to achieve the desired data format, we perform the following steps:

1. Create a list of scores, including cumulative scores in ascending order.
2. Generate a corresponding list of locations.

Additionally, you can to choose whether to include correction for day occurrence in the data. (The code below uses the scores with occurrence correction)


In [9]:
def get_score_and_location_lists(sums_df, unique_candidates_df=unique_candidates):
    """
    Convert a matrix (sums_df) containing scores for selected locations into two lists:
    
    1. A score list: A list of scores for each selected location.
    2. A location list: A list of tuples, each containing the x and y coordinates of a selected location.

    Args:
        sums_df (DataFrame): A DataFrame containing the scores for selected locations.
        unique_candidates_df (DataFrame, optional): A DataFrame with information about the selected locations. Defaults to 'unique_candidates'.

    Returns:
        list: A list of scores for each selected location.
        list: A list of location tuples (x, y) for each selected location.
    """
    sum_df = sums_df.reset_index()
    sum_df.columns = ['Index', 'Sum']

    results_df = unique_candidates_df.copy()
    combined_df = pd.concat([results_df, sum_df], axis=1)
    combined_df = combined_df.sort_values(by='Sum', ascending=False)
    combined_df = combined_df.drop(columns='Index')

    score_list = combined_df['Sum'].tolist()
    location_df = combined_df[['first_point_x', 'first_point_y']]
    location_list = list(location_df.to_records(index=False))
    return score_list, location_list

In [10]:
score_list, location_list = get_score_and_location_lists(weights)

## Selecting the Top 120 Locations

In this step, we aim to identify the best locations among a list of candidates, each assigned a score. Our goal is to select the top 120 locations with the highest scores. To ensure a diverse and effective selection, we have applied specific constraints:

1. **Minimum Euclidean Distance:** We impose a constraint that the Euclidean distance between selected inspector locations must exceed a minimum threshold.

2. **Minimum Path Distance:** Another constraint we enforce is that the path distance between selected locations must be greater than a specified minimum value.

These constraints help prevent the selection of closely located locations with high scores, ultimately leading to a more balanced and efficient choice of inspector locations.

Additionally, it's worth noting that the number of inspectors can be easily adjusted by changing this parameter. This flexibility allows for the optimization of the inspector team size based on specific needs and requirements.



In [11]:
def find_closest_node(graph, target_node):
    """
    Find the closest matching node in a given graph for a target node name.

    This function calculates the similarity between the target node name and each node in the graph
    and returns the node with the highest similarity score.

    Args:
        graph (networkx.Graph): The graph containing nodes to search for a match.
        target_node (str): The target node name for which to find the closest match.

    Returns:
        str: The node name from the graph that best matches the target node name.
    """
    closest_node = None
    max_similarity = -1

    for node in graph.nodes:
        similarity = fuzz.token_sort_ratio(target_node, node)
        if similarity > max_similarity:
            closest_node = node
            max_similarity = similarity

    return closest_node


In [12]:
def find_optimal_locations(G, score_list, location_list, max_range, max_direct_range, number_of_inspectors):
    """
    Selects a specified number of locations with the highest scores while considering constraints on minimum Euclidean and direct distances.


    Args:
        G (networkx.Graph): The graph representing the area where inspection locations are selected.
        score_list (list): A list of scores for each location.
        location_list (list): A list of location coordinates (x, y).
        max_range (float): The maximum allowed Euclidean distance between selected locations.
        max_direct_range (float): The maximum allowed direct (straight-line) distance between selected locations.
        number_of_inspectors (int): The desired number of inspection locations to select.

    Returns:
        list: A list of selected optimal inspection locations, each represented as a tuple (x, y).
        list: A list of scores corresponding to the selected locations.
    """
    final_locations = []
    Sum = []

    for item in range(len(score_list)):
        extent = location_list[item]

        if not str(extent) in G:
            extent = find_closest_node(G, extent)
            extent = ast.literal_eval(extent)

        costs = [1e6]
        direct_costs = [1e6]

        for loc in final_locations:
            direct_cost = ((loc[0] - extent[0])**2 + (loc[1] - extent[1])**2) ** (1/2)
            direct_costs.append(direct_cost)

            try:
                cost = nx.shortest_path_length(G, str(extent), str(loc), weight='weight')
            except:
                cost = 1e6
            costs.append(cost)

            try:
                cost = nx.shortest_path_length(G, str(loc), str(extent), weight='weight')
            except:
                cost = 2e6
            costs.append(cost)

        min_cost = min(costs)
        min_direct_cost = min(direct_costs)

        if (min_cost > max_range) and (min_direct_cost > max_direct_range):
            final_locations.append(extent)
            Sum.append(score_list[item])
        # else:
        #     print('Location is too close')

        if len(final_locations) == number_of_inspectors:
            break

    return final_locations, Sum


In [13]:
def process_final_locations(final_locations, Sum):
    """
    Convert lists of final inspection locations and corresponding scores into a DataFrame.

    Args:
        final_locations (list of tuples): A list of tuples representing inspection locations with (x, y) coordinates.
        Sum (list): A list of scores corresponding to each inspection location.

    Returns:
        pd.DataFrame: A DataFrame containing columns for 'X_value' (x-coordinate), 'Y_value' (y-coordinate), and 'Sum_weight' (score).
    """
    data = final_locations
    x, y = zip(*data)
    x_values = list(x)
    y_values = list(y)
    
    # # Assuming you have the 'DutchRDtoWGS84' function defined elsewhere
    # wgs_coords = [functions.DutchRDtoWGS84(x, y) for x, y in zip(x_values, y_values)]
    # wgs_x_values, wgs_y_values = zip(*wgs_coords)
    
    Inspector_locations = pd.DataFrame({'X_value': x_values, 'Y_value': y_values, 'Sum_weight': Sum})
    display(Inspector_locations)
    return Inspector_locations

In [14]:
min_direct_range = 3000
min_range = 10000
number_of_inspectors = 120
final_locations, Sum = find_optimal_locations(G, score_list, location_list, min_range, min_direct_range, number_of_inspectors)
Inspector_locations = process_final_locations(final_locations, Sum)

,X_value,Y_value,Sum_weight
0,157715.96900,379454.41900,0.082284
1,119810.34000,493682.34000,0.066861
2,85860.07700,432976.59800,0.065860
3,108158.90600,450448.65000,0.062600
4,95910.01500,462530.83800,0.059399
...,...,...,...
115,128428.44700,394457.72100,0.020799
116,226915.85092,579821.64526,0.020633
117,100002.90400,459765.09600,0.020623
118,171203.31600,482428.90300,0.019934


## Creating Clones

To evaluate the performance of the selected locations, we employ a unique approach: the creation of mirrored clones on the opposite side of the road. This mirroring process is essential because the road network lacks the necessary data to facilitate straightforward navigation of off and on ramps for travel in the opposite direction.

The procedure for creating these clones involves determining the direction of the road using the RPE_code, which can be 'L' (left), 'R' (right), or '#' (indicating bidirectional roads). Based on the RPE_code, a new network is generated with the opposite road directions, and the nearest node in this new network is identified.


In [15]:
def other_road_side(node, G):
    """
    Create a new graph containing only nodes on the opposite road side.

    This function takes a node in a graph (G) and identifies its corresponding road side (RPE_CODE). It then creates a new graph (RPE_G) that includes only nodes with the opposite road side value, such as 'L' for 'R' or 'R' for 'L'. If the node's road side is '#' or if there are no nodes on the opposite road side, it returns None.

    Args:
        node: A node in the input graph G.
        G (networkx.Graph): The original graph containing road-side information.

    Returns:
        networkx.Graph or None: A new graph containing nodes on the opposite road side or None if there are no such nodes.
    """
    RPE = G.nodes[node]['RPE_CODE']
    if RPE == "L":
        value_to_match = "R"
    if RPE == "R":
        value_to_match = "L"
    if RPE == "#":
        return None
    selected_nodes = []
    for node in G.nodes:
        RPE = G.nodes[node]['RPE_CODE']
        if RPE == value_to_match:
            selected_nodes.append(node)
    RPE_G = nx.Graph()
    RPE_G.add_nodes_from(selected_nodes)
    return RPE_G

In [16]:
def find_closest_node_other_side(graph, x1, y1):
    """
    Find the closest node in a given graph based on Euclidean distance. (Used for finding the closest node on the other side of the road)

    This function calculates the Euclidean distance between a specified point (x1, y1) and each node in the graph. It returns the node from the graph that is closest to the specified point, along with the calculated minimum distance.

    Args:
        graph (networkx.Graph): The graph containing nodes to search for the closest node.
        x1 (float): The x-coordinate of the specified point.
        y1 (float): The y-coordinate of the specified point.

    Returns:
        str: The node in the graph that is closest to the specified point.
        float: The minimum Euclidean distance between the specified point and the closest node.
    """
    closest_node = None
    min_distance = float('inf')
    for node in graph.nodes:
        x2, y2 = functions.parse_coordinate(node)
        distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

        if distance < min_distance:
            closest_node = node
            min_distance = distance

    return closest_node, min_distance


In [17]:
def create_clones(graph, node, max_range = 500):
    """
    Create clone inspector locations on the opposite side of the road for a selected inspector location.

    Args:
        graph (networkx.Graph): The graph containing road-side information and nodes.
        node: The selected inspector location node.
        max_range (float): The maximum range within which the clone inspector location must be located on the opposite side of the road. Defaults to 500.

    Returns:
        tuple or None: A tuple representing the clone inspector location (x, y) if within max_range, or None if not.
    """
    node = find_closest_node(graph, str(node))
    x, y = functions.parse_coordinate(node)
    RPE_G = other_road_side(node, graph)
    if RPE_G == None:
        return None
    RPE_node, RPE_sim = find_closest_node_other_side(RPE_G, x, y)
    if RPE_sim < max_range:
        x_str, y_str = str(RPE_node).strip('()').split(', ')
        RPE_node_x, RPE_node_y = (float(x_str), float(y_str))
        return (RPE_node_x, RPE_node_y)  
    else:
        return None

In [18]:
Inspector_locations['Other_side'] = Inspector_locations.apply(lambda row: create_clones(G, f"({row['X_value']}, {row['Y_value']})"), axis=1)
display(Inspector_locations)

,X_value,Y_value,Sum_weight,Other_side
0,157715.96900,379454.41900,0.082284,"(157697.714, 379481.622)"
1,119810.34000,493682.34000,0.066861,"(119834.0, 493703.0)"
2,85860.07700,432976.59800,0.065860,"(85885.947, 432987.14)"
3,108158.90600,450448.65000,0.062600,"(108230.069, 450427.745)"
4,95910.01500,462530.83800,0.059399,"(95915.967, 462499.839000001)"
...,...,...,...,...
115,128428.44700,394457.72100,0.020799,"(128696.786, 394449.985)"
116,226915.85092,579821.64526,0.020633,"(226911.18616, 579840.54611)"
117,100002.90400,459765.09600,0.020623,"(100187.217, 459668.969)"
118,171203.31600,482428.90300,0.019934,"(171184.947, 482445.059)"


# Calculate Performance of Selected Locations

The following code is an adaptation of the optimization model. It uses only the 120 chosen locations as input, rather than all the candidate points. The goal is to optimize the assignment of road inspectors to incidents, minimizing the total distance while satisfying capacity and service constraints.

As the core process aligns with the optimization model, detailed explanations are omitted here and can be found in the "Optimization Model" notebook. The code is executed for a single incident day to reduce computation time.

(Numerical results can be found at the bottom)


In [19]:
def filter_incidents(select_day, inc_df):
    """
    Filter and preprocess incidents for a selected day from the total incidents DataFrame.

    Args:
        select_day (datetime.date): The date for which incidents will be filtered.
        inc_df (pd.DataFrame): The total incidents DataFrame containing incident data.

    Returns:
        pd.DataFrame: A filtered and preprocessed DataFrame with incidents that occurred on the selected day. The DataFrame is prepared for future calculations with additional columns.
    """
    
    # filter to incidents which happened on selected_day
    filt_inc = inc_df[inc_df['starttime_new'].dt.date == select_day]
    filt_inc = filt_inc.copy()

    # add dutch rd coordinates
    DutchRD = filt_inc.apply(lambda x: functions.WGS84toDutchRD(x.primaire_locatie_lengtegraad, 
                                                                x.primaire_locatie_breedtegraad), axis=1)
    filt_inc['dutch_rd_x'] = DutchRD.apply(lambda x: x[0])
    filt_inc['dutch_rd_y'] = DutchRD.apply(lambda x: x[1])

    # add columns to the DF which will be used later
    filt_inc['Node'] = None
    filt_inc['Node_distance_to_incident'] = None
    filt_inc['Node_distance_line'] = None
    filt_inc['Node_line'] = None
    filt_inc['Node_RPE'] = None

    return filt_inc

In [20]:
def find_nearest_node_incidents_new(x, y, gdf, tolerance=25):
    """
    Find the nearest linestring in a GeoDataFrame to a given point (x, y) and calculate distances.

    This function takes a point with coordinates (x, y) and a GeoDataFrame (gdf) containing linestrings. It determines which linestring from the network is located closest to the specified point and calculates the distance to that linestring. It also checks if the point is within a specified tolerance distance to the nearest linestring.

    Args:
        x (float): The x-coordinate of the point.
        y (float): The y-coordinate of the point.
        gdf (geopandas.GeoDataFrame): The GeoDataFrame containing linestrings.
        tolerance (float, optional): The tolerance distance within which the point is considered to be on the nearest linestring. Defaults to 25.

    Returns:
        dict: A dictionary containing information about the nearest linestring and distances, including:
            - 'Node': The nearest node coordinates.
            - 'Line': The nearest linestring.
            - 'RPE': The road-side information of the nearest linestring.
            - 'Total Distance to Line': The total distance to the nearest linestring.
            - 'Distance to Incident': The Euclidean distance from the point to the nearest node on the linestring.
            If the point is not within the tolerance distance, the dictionary values are set to None.
    """
    point = f"POINT ({x} {y})"
    
    # create a gdf of the point and all lines in the network
    gdf_line = gpd.GeoDataFrame(gdf['geometry'])
    
    # crs is determined by the crs of the linestrings in the network
    gdf_point = gpd.GeoDataFrame(geometry=[shapely.wkt.loads(point)], crs=gdf_line.crs)
    
    # finds the nearest linestring and merges the two gdf's together
    df_n = gpd.sjoin_nearest(gdf_point, gdf_line).merge(gdf_line, left_on="index_right", right_index=True)
    
    # calculates the distance from the point to the linestring
    df_n['distance'] = df_n.apply(lambda r: r["geometry_x"].distance(r["geometry_y"]), axis=1)
    
    # if two or more linestrings are at an equal distance from the point it only considers the first linestring
    if len(df_n) > 1:
        df_n = pd.DataFrame(df_n.iloc[0]).T
        df_n = gpd.GeoDataFrame(df_n, geometry='geometry_y', crs=gdf_line.crs)
    
    # checks whether the point is located within the tolarance distance
    if df_n['distance'].min() < tolerance:
        
        # I copied this from Klaas, I'm not sure why he takes the last coordinate
        nearest_node = df_n['geometry_y'].item().coords[-1]
        nearest_line = df_n['geometry_y'].item()
        nearest_RPE = gdf.iloc[df_n['index_right']]['RPE_CODE'].item()
        total_distance_to_nearest_line = gdf.iloc[df_n['index_right']]['ENDAFSTAND'].item()
        distance_to_incident = np.sqrt((nearest_node[0] - x) ** 2 + (nearest_node[1] - y) ** 2)

        return {'Node': nearest_node,
                'Line': nearest_line,
                'RPE': nearest_RPE,
                'Total Distance to Line': total_distance_to_nearest_line,
                'Distance to Incident': distance_to_incident}
    
    # if the point is not within the tolarance distance it returns an empyt dict
    else:
        return {'Node': None,
                'Line': None,
                'RPE': None,
                'Total Distance to Line': None,
                'Distance to Incident': None}

In [21]:
def calculate_values(row):
    """
    Calculate and store values related to the nearest node and linestring for an incident.

    Args:
        row (pd.Series): A row from the incidents DataFrame containing incident data.
        gdf (geopandas.GeoDataFrame): The GeoDataFrame containing linestrings for the network.

    Returns:
        pd.Series: The input row with additional columns storing calculated values, including:
            - 'Node': The nearest node coordinates.
            - 'Node_distance_line': The total distance to the nearest linestring.
            - 'Node_line': The nearest linestring.
            - 'Node_distance_to_incident': The Euclidean distance from the incident to the nearest node.
            - 'Node_RPE': The road-side information of the nearest linestring.
    """

    x = row['dutch_rd_x']
    y = row['dutch_rd_y']
    result = find_nearest_node_incidents_new(x, y, gdf)
    
    row["Node"] = result['Node']
    row["Node_distance_line"] = result['Total Distance to Line']
    row["Node_line"] = result['Line']
    row['Node_distance_to_incident'] = result['Distance to Incident']
    row['Node_RPE'] = result['RPE']
    
    return row

In [22]:
def drop_inc_out_of_range(inc_df, print_bool=False):
    """
    Drop incidents that are more than 25 meters from the nearest network link.

    Args:
        inc_df (pd.DataFrame): The incidents DataFrame.
        print_bool (bool, optional): If True, the function prints the percentage of dropped rows. Defaults to False.

    Returns:
        pd.DataFrame: A DataFrame containing incidents that are within 25 meters of the nearest road network link.
    """
    
    nan_count = inc_df['Node'].isna().sum()
    if print_bool:
        print(f'Percentage of dropped rows: {(nan_count / len(inc_df) * 100):.2f}%, '+
               'because these incidents are more than 25 meters from the closest road.')

    # all incidents which are too far away from the network are dropped
    incidents = inc_df.dropna().reset_index(drop=True)
    
    return incidents

In [23]:
def process_inspectors_data(inspectors_df, G):
    """
    Process inspector location data and update it with nearest road nodes.

    Args:
        inspectors_df (pd.DataFrame): DataFrame containing inspector location data.
        G (networkx.Graph): Graph representing road networks for finding nearest road nodes.

    Returns:
        pd.DataFrame: Processed inspector location data with updated coordinates.
    """
    # Rename and select relevant columns
    inspectors_df.rename(columns={'Other_side': 'Clone'}, inplace=True)
    inspectors_df = inspectors_df[['X_value', 'Y_value', 'Clone']]

    # Convert all coordinates to string
    inspectors_df['Main'] = inspectors_df.apply(lambda row: str((row['X_value'], row['Y_value'])), axis=1)
    inspectors_df = inspectors_df[['Main', 'Clone']]

    # Add the coordinates from the find_closest_node function
    inspectors_df['Main_updated'] = inspectors_df['Main'].apply(lambda x: find_closest_node(G, x))
    inspectors_df['Clone_updated'] = inspectors_df['Clone'].apply(lambda x: find_closest_node(G, x))
    
    return inspectors_df

In [24]:
def create_empty_cost_M(uni_cand, inc_df, G):
    """
    Create an empty cost matrix for candidate and incident nodes.

    Args:
        uni_cand (pd.DataFrame): A DataFrame containing candidate node information, including 'Main_updated' and 'Clone_updated'.
        inc_df (pd.DataFrame): The incidents DataFrame.
        G (networkx.Graph): The graph containing road network information.

    Returns:
        pd.DataFrame: An empty cost matrix with candidate nodes as rows and incident nodes as columns.
        list: The column names (incident nodes) extracted from the incidents DataFrame and present in the graph 'G'.
    """

    num_rows = len(uni_cand)
    placeholder_value = None  # You can use np.nan or any other value as a placeholder
    col_names = [str(node) for node in list(inc_df['Node']) if str(node) in G.nodes()]
    main_names = uni_cand['Main_updated'].values
    clone_names = uni_cand['Clone_updated'].values
    num_columns = len(col_names)
    data = [[placeholder_value] * num_columns for _ in range(num_rows)]
    cost_M = pd.DataFrame(data, columns=col_names)
    cost_M.insert(0, 'Main', main_names)
    cost_M.insert(1, 'Clone', clone_names)
    
    return cost_M, col_names

In [25]:
def parse_coordinate(coordinate_string):
    """
    Parse a string of coordinate values to obtain usable x and y coordinates.

    Args:
        coordinate_string (str): A string containing the coordinate values in the format "(x, y)".

    Returns:
        tuple: A tuple containing the parsed x and y coordinates as floats.
    """
    # Remove parentheses and split the string by comma
    if coordinate_string[-2] == '.':
        coordinate_string = coordinate_string[:-2]
    cleaned_string = coordinate_string.strip('()')
    x_str, y_str = cleaned_string.split(',')

    # Convert the string values to floats
    x = float(x_str.strip())
    y = float(y_str.strip())

    return x, y

In [26]:
def calculate_cost_matrix(cost_M, G, print_bool=False):
    """
    Fill in all values of the cost matrix based on network distances.

    This function populates the cost matrix with values representing distances or costs between candidate nodes and incident nodes. It uses network-based distances where possible, including Euclidean distance and shortest path lengths, while considering constraints for large distances.

    Args:
        cost_M (pd.DataFrame): An empty cost matrix with candidate nodes as rows and incident nodes as columns.
        G (networkx.Graph): The graph containing road network information.
        print_bool (bool, optional): If True, the function prints the time taken for the calculations. Defaults to False.

    Returns:
        pd.DataFrame: The filled cost matrix with calculated distances or costs between candidate and incident nodes.
    """
    
    t0 = time.time()
    cost_M = cost_M.copy()
    no_counter = 0
    
    for row_idx in range(cost_M.shape[0]):

        main_node = cost_M.iloc[row_idx, 0]
        clone_node = cost_M.iloc[row_idx, 1]

        main_node_x, main_node_y = parse_coordinate(main_node)
        
        for col_idx in np.arange(2, cost_M.shape[1]):

            # store coordinates of incident node as tuple
            goal_node = parse_coordinate(cost_M.columns[col_idx])

            # calculate euclidean distance
            dx = main_node_x - goal_node[0]
            dy = main_node_y - goal_node[1]
            distance = np.sqrt(dx*dx + dy*dy)

            # if euclidean distance is larger than 50 km set a large value as cost
            if distance > 50000:
                cost = 1e9

            # otherwise try to calculate shortest path
            else:
                try:
                    cost_main = nx.shortest_path_length(G, str(main_node), str(goal_node), 
                                                   weight='weight')
                    try:
                        cost_clone = nx.shortest_path_length(G, str(clone_node), str(goal_node), 
                                                       weight='weight')
                    except:
                        cost = cost_main
                    else:
                        # add 2 km to the cost of the clone node and then take min cost
                        cost = np.min([cost_main, cost_clone + 2000])

                # if shortest path still throws an error then store other large value
                except:
                    no_counter += 1
                    cost = 2e9

            # store calculated cost in the correct postion in matrix
            cost_M.iloc[row_idx, col_idx] = cost
            
    t1 = time.time()

    total = t1-t0
    if print_bool:
        print(f'calculating the whole cost matrix took: {total/60:.2f} minutes')
        
    return cost_M

In [27]:
def drop_unconnected_incidents(cost_M, print_bool=False):
    """
    Remove incidents with excessively large costs from the cost matrix.

    This function removes incidents from the cost matrix if the cost to the nearest candidate node is larger than 1,000 kilometers. 

    Args:
        cost_M (pd.DataFrame): The cost matrix with candidate nodes as rows and incident nodes as columns.
        print_bool (bool, optional): If True, the function prints the percentage of dropped incidents. Defaults to False.

    Returns:
        pd.DataFrame: The cost matrix with unconnected incidents removed.
    """
    cost_M = cost_M.copy()
    
    # get the minimum cost values for each incident node
    min_values = cost_M.iloc[:, 2:].min()
    columns_to_drop = []

    # iterate over al min costs and if this is larger than 1000 km add it to the drop list
    for col, val in min_values.items():
        if pd.notna(val) and val > 1e6:
            columns_to_drop.append(col)

    cost_M.drop(columns=columns_to_drop, inplace=True)
    if print_bool:
        print(f'{(len(columns_to_drop) / len(min_values) * 100):.2f}% of the incidents are '+
              'dropped because there is no route possible from any candidate node')
    
    return cost_M

In [28]:
def subtract_dist_to_node(cost_M, inc):
    """
    Adjust the cost matrix by subtracting the distance from incidents to their nearest road nodes.

    Args:
        cost_M (pd.DataFrame): A cost matrix with distances or costs between candidate nodes and incidents.
        inc (pd.DataFrame): A DataFrame containing information about incidents, including incident locations and distances to the nearest road nodes.

    Returns:
        pd.DataFrame: An adjusted cost matrix with reduced costs after subtracting distances to nearest road nodes.
        list: A list of incident locations (Dutch RD coordinates).
    """
    incidents_loc = []
    distance_store = []
    result_df = cost_M.copy()  # Create a copy of the DataFrame

    # iterate over all incident nodes
    for name in cost_M.iloc[:, 2:].columns:
        if name[-2] == '.':
            count = int(name[-1])
            N = name[:-2]
        else:
            count = 0
            N = name

        # get the data for the incident node
        match = inc[inc["Node"].astype(str) == N]

        # get the distance from the incident to the end node
        dist = match['Node_distance_to_incident'].values
        distance_to_node = dist[count]

        # get the dutch rd coordinates of the incident
        incident_loc = (match["dutch_rd_x"].values[0], match["dutch_rd_y"].values[0])

        incidents_loc.append(incident_loc)
        distance_store.append(distance_to_node)

    result_df.iloc[:, 2:] -= distance_store
    result_df.iloc[:, 2:] = result_df.iloc[:, 2:].abs()
    
    return result_df, incidents_loc

In [29]:
def store_cost_matrix_in_arrays(cost_M):
    """
    Store the values in the cost matrix in arrays for candidates and incidents.

    Args:
        cost_M (pd.DataFrame): The cost matrix with candidate nodes as rows and incident nodes as columns.

    Returns:
        np.ndarray: The cost matrix values for distances or costs between candidates and incidents.
        np.ndarray: The coordinates of candidate nodes.
        np.ndarray: The coordinates of incident nodes.
    """
    # store the values in arrays
    cost_matrix = np.array(cost_M.iloc[:, 2:].values)
    final_incidents_nodes = np.array(cost_M.columns[2:])
    final_candidates = np.array(cost_M['Main'].values)

    # store the converted coordinate values into a new array
    candidates = np.array([parse_coordinate(node) for node in final_candidates])
    incidents = np.array([parse_coordinate(node) for node in final_incidents_nodes])
    
    return cost_matrix, candidates, incidents

In [30]:
def run_gurobi_model(selected_day, cost_M, cand_arr, inc_arr, print_bool=False):
    """
    Create a Gurobi optimization model for assigning inspectors to incidents.

    This function sets up a Gurobi optimization model, adds the objective function and constraints, and then runs the optimization to determine the assignment of inspectors to incidents. It returns the results, including the selected inspector nodes and a binary matrix indicating the assignment of inspectors to incidents.

    Args:
        selected_day (datetime.date): The date for which the optimization is performed.
        cost_M (np.ndarray): A cost matrix with distances or costs between candidate nodes and incidents.
        cand_arr (np.ndarray): Coordinates of candidate nodes.
        inc_arr (np.ndarray): Coordinates of incident nodes.
        print_bool (bool, optional): If True, the function prints summary results. Defaults to False.

    Returns:
        list: A list of results including the selected day, number of inspectors, number of incidents,
              objective function value, average response time (in seconds), and average response time (in minutes).
        list: A list of selected inspector nodes.
        np.ndarray: A binary matrix indicating the assignment of inspectors to incidents.
    """
    # initialise model
    model = Model("Road inspectors")
    
    # Setting up the indices
    N = np.arange(cand_arr.shape[0])
    K = np.arange(inc_arr.shape[0])

    # Parameters
    inspector_cap = 10 # the maximum amount of incidents an inspector can handle in a day
    N_inspectors = 120 # the maximum amount of inspectors
    speed = 100 # a constant speed value in km/h

    if inspector_cap * N_inspectors < inc_arr.shape[0]:
        raise ValueError("Not enough inspectors!")
        
    pair = [(i, k) for i in N for k in K]

    # Setting up the variables
    x = model.addVars(pair, vtype=GRB.BINARY, name="x")
    c = model.addVars(N, vtype=GRB.BINARY, name="c")

    model.Params.MIPGap = 0.01 # set the threshold value at which the optimisation stops to 1.0%
    model.Params.IntegralityFocus=0
    model.update()
    
    # Objective function: Minimise the total travel cost
    model.setObjective(quicksum(cost_M[i, k] * x[i, k] for i in N for k in K),
                       GRB.MINIMIZE)
    
    # Constraints: All incidents must be handled
    model.addConstrs(quicksum(x[i, k] for i in N) == 1 for k in K)
    
    # Constraints: Inspector capacity
    model.addConstrs(quicksum(x[i, k] for k in K) <= (inspector_cap * c[i]) for i in N)
    
    # Constraints: Number of inspectors
    model.addConstr(quicksum(c[i] for i in N) <= N_inspectors)
    
    # Run the model
    model.update()
    model.optimize()
    
    # Collect the results
    inspectors = []



    # Collect the matching
    match_arr = np.zeros((len(N), len(K)))
    for i in N:
        for k in K:
            match_arr[i, k] = x[i, k].x

    for i in N:
        if c[i].x == 1:
            inspectors.append(i)
    
    # Calculate the average response time in seconds
    ave_time = model.ObjVal / len(inc_arr) / 1000 / speed * 3600
    # Collect objective date, number of inspectors, number of incidents,
    # objective function values, average response time[sec], average response time [min]
    output = [selected_day, len(N), len(K),
               model.ObjVal, ave_time, ave_time/60]
            
    # Summary of results
    if print_bool:
        print(f'Minimum total cost is {model.ObjVal/1000:.2f} km')
        print(f'Number of inspectors is {len(inspectors)}')
        print(f'The distance per inspector is {model.ObjVal / len(inspectors) / 1000 :.2f} km')

        
        print(f'Average time to reach the incident is {ave_time//60:.0f} minutes and {ave_time%60:.0f} seconds')
    
    return output, inspectors, match_arr

In [31]:
def run_everything_for_optimisation (selected_day, all_incidents, inspectors_df, G):
    """
    Run all calculations for the optimization model.

    This function performs a series of calculations and optimization steps for the road inspector allocation model based on the following input parameters:
    
    Args:
        selected_day (datetime.date): The day for which calculations are performed.
        all_incidents (pd.DataFrame): DataFrame containing information about all incidents.
        inspectors_df (pd.DataFrame): DataFrame with inspector location data.
        G (networkx.Graph): Network graph representing road networks.
        input_day (str): The input day used in file naming or other purposes.

    Returns:
        tuple: A tuple containing the following elements:
            - output (list): Results including the number of inspectors, incidents, objective value, average response time in seconds, and average response time in minutes.
            - match_arr (numpy.ndarray): A 2D array representing the assignment of inspectors to incidents.
            - incidents_location (pd.DataFrame): DataFrame containing incident locations.
            - inspectors (list): List of inspector locations selected by the optimization model.
    """
    inspectors_df = process_inspectors_data(inspectors_df, G)
    
    filtered_incidents = filter_incidents(selected_day, all_incidents)

    incidents_results = filtered_incidents.apply(calculate_values, axis=1)
    
    incidents = drop_inc_out_of_range(incidents_results)

    cost_matrix, column_names = create_empty_cost_M(inspectors_df, incidents, G)

    fill_cost_matrix = calculate_cost_matrix(cost_matrix, G)

    clean_cost_matrix = drop_unconnected_incidents(fill_cost_matrix)
    
    end_cost_matrix, incidents_loc = subtract_dist_to_node(clean_cost_matrix, incidents)
    
    incidents_location = pd.DataFrame(incidents_loc, columns=['x', 'y'])

    cost_matrix, candidates, incidents = store_cost_matrix_in_arrays(end_cost_matrix)

    if cost_matrix.shape[0] != candidates.shape[0] or cost_matrix.shape[1] != incidents.shape[0]:
        raise ValueError("Dimensions do not match")

    output, inspectors, match_arr = run_gurobi_model(selected_day, cost_matrix, candidates, incidents, print_bool=True)

    return output, match_arr, incidents_location, inspectors
    

In [32]:
day = '2019-07-31'
selected_day = datetime.strptime(day, '%Y-%m-%d').date()
output, match_arr, incidents_location, inspectors = run_everything_for_optimisation(selected_day, all_incidents, Inspector_locations, G)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-10
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 369 rows, 29880 columns and 59760 nonzeros
Model fingerprint: 0xbc7ccb80
Variable types: 0 continuous, 29880 integer (29880 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+02, 2e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.890039e+11
Presolve removed 1 rows and 120 columns
Presolve time: 0.08s
Presolved: 368 rows, 29760 columns, 59520 nonzeros
Variable types: 0 continuous, 29760 integer (29760 binary)
Found heur

In [33]:
print("Calibration results for:", output[0])
print()
print("Minimum Total Cost:", f"{output[3] / 1000:.2f}", "km")
print("Number of Inspectors:", len(inspectors))
print("Distance per Inspector:", f"{output[3] / len(inspectors) / 1000 :.2f}", "km")
print("Average Time to Reach Incidents:", f"{output[4]//60:.0f} minutes and {output[4]%60:.0f} seconds")

Calibration results for: 2019-07-31

Minimum Total Cost: 2327.26 km
Number of Inspectors: 120
Distance per Inspector: 19.39 km
Average Time to Reach Incidents: 5 minutes and 38 seconds
